In [1]:
about_the_run = "About the run: "
run, load_run_number = 2, 66             #  0:"single_run"  1:"keras_tuner" 2:"load_best_model"
scenario = "rcp26"
future_type= "_far_future"   #options: "_near_future", "_mid_future", "_far_future"
percentage = 100
load_run_number = 66
colab = 0
model_choice = "tf"
percentage = 1
epochs = 30
batch_size=32
limit_of_nans_in_a_timestep = 120
percentage_of_data_in_summer_months = [0.4]  # Desired number of zero values
divide_latitude_in_these_many_parts = 5  #latitude has 10 values
divide_longitude_in_these_many_parts = 6 # longitude has 14 values
climate_data_folder_name = f"{scenario}{future_type}"

# Configurations

In [3]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import plotly.express as px
import math
import time
from sklearn.linear_model import LinearRegression
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from kerastuner.tuners import RandomSearch
import logging
import os
import sys
from contextlib import contextmanager
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import optimizers
from keras.models import load_model
import contextlib
import sys

C:\Users\suvam\AppData\Local\Temp\ipykernel_22404\3295121848.py:15: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
# Define the folder paths
ig_stuff_folder = 'ig_stuff'
images_folder = os.path.join(ig_stuff_folder, 'images')

# Check if the ig_stuff folder exists, if not, create it
if not os.path.exists(ig_stuff_folder):
    os.makedirs(ig_stuff_folder)
    print(f"Created folder: {ig_stuff_folder}")

# Check if the images folder exists inside ig_stuff, if not, create it
if not os.path.exists(images_folder):
    os.makedirs(images_folder)
    print(f"Created folder: {images_folder}")

print("Directory structure is set up.")

In [4]:
def get_next_folder_name(base_folder):
    run_number = 1
    while True:
        folder_name = os.path.join(base_folder, f"run_{run_number}")
        if not os.path.exists(folder_name):
            return folder_name
        run_number += 1

# Base folder directory
base_directory = r".\model_runs"

# Get the next folder name
folder_name = get_next_folder_name(base_directory)

# Create the folder
os.makedirs(folder_name, exist_ok=True)

run_number = f"run_{folder_name.split('_')[2]}"
print(run_number)

run_78


In [5]:
# Set up logging and save the log file in the folder
log_file = os.path.join(folder_name, 'log.txt')
logger = logging.getLogger(folder_name)
logger.setLevel(logging.INFO)

# Create a file handler and set the formatter
file_handler = logging.FileHandler(log_file)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

# Add the file handler to the logger
logger.addHandler(file_handler)

# Create a stream handler to display log messages in Jupyter Notebook console
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

# Log information
logger.info("Logging information check")
logger.info(run_number)

2024-05-24 15:59:26,979 - INFO - Logging information check
2024-05-24 15:59:26,981 - INFO - run_78


In [6]:
logger.info(f"About the run: {about_the_run}")
logger.info(f"run = {run}   =>  0:single_run  1:keras_tuner 2:load_best_model")
logger.info(f"model_choice ={model_choice}")
logger.info(f"percentage = {percentage}")
logger.info(f"epochs = {epochs}")
logger.info(f"batch_size = {batch_size}")
logger.info(f"percentage_of_data_in_summer_months = {percentage_of_data_in_summer_months}")
logger.info(f"divide_latitude_in_these_many_parts = {divide_latitude_in_these_many_parts}")  
logger.info(f"divide_longitude_in_these_many_parts = {divide_longitude_in_these_many_parts}") 


2024-05-24 15:59:27,017 - INFO - About the run: About the run: 
2024-05-24 15:59:27,020 - INFO - run = 2   =>  0:single_run  1:keras_tuner 2:load_best_model
2024-05-24 15:59:27,022 - INFO - model_choice =tf
2024-05-24 15:59:27,025 - INFO - percentage = 1
2024-05-24 15:59:27,027 - INFO - epochs = 30
2024-05-24 15:59:27,028 - INFO - batch_size = 32
2024-05-24 15:59:27,030 - INFO - percentage_of_data_in_summer_months = [0.4]
2024-05-24 15:59:27,032 - INFO - divide_latitude_in_these_many_parts = 5
2024-05-24 15:59:27,034 - INFO - divide_longitude_in_these_many_parts = 6


In [7]:
if colab == 1:
    # !pip install cftime
    from google.colab import drive
    drive.mount('/content/drive')
    %run "/content/drive/My Drive/Colab Notebooks/main/functions.ipynb"
else: 
    %run "functions.ipynb"

# Data Loading

In [8]:
# Step 1: Load NDSI Labels
if colab !=1:
  ndsi_ds = xr.open_mfdataset(r".\cropped_data\label.nc")

if colab == 1:
  ndsi_ds = xr.open_mfdataset(f'/content/drive/My Drive/Colab Notebooks/cropped_data/label.nc')

# filtered_dates, nan_interpolation_df = get_filtered_dates_for_ndsi(percentage_limit=percentage_limit, pixel_interpolation_limit=pixel_interpolation_limit)
# ndsi_ds = ndsi_ds.sel(time=filtered_dates)
selected_dates = get_dates(ndsi_ds)
ndsi_labels = ndsi_ds['NDSI_Snow_Cover'].values
ndsi_ds = ndsi_ds.interpolate_na(dim='lon', method='linear',  max_gap=4, use_coordinate=False)
ndsi_ds = ndsi_ds.interpolate_na(dim='lat', method='linear', max_gap=3, use_coordinate=False)

ndsi_ds['time'] = xr.DataArray(ndsi_ds['time'].values.astype('datetime64[ns]'), dims='time', attrs=ndsi_ds['time'].attrs)
ndsi_ds.close()

In [9]:
# Step 2: Load Data
# Load climate variables
climate_vars = ["tas", "pr", "hurs", "psl",  "rsds", "sfcWind"]
data = []

for parameter in climate_vars:
    
    if colab == 0:
      ds = xr.open_mfdataset(rf'.\cropped_data\{climate_data_folder_name}\{parameter}_{scenario}.nc') #fixxx
    if colab == 1:
      ds = xr.open_mfdataset(f'/content/drive/My Drive/Colab Notebooks/cropped_data/{climate_data_folder_name}/{parameter}_{scenario}.nc')

    data.append(ds[parameter].values)
    ds.close()

data = np.array(data)

In [10]:
ds

<xarray.Dataset>
Dimensions:  (time: 3652, y: 10, x: 14)
Coordinates:
  * time     (time) datetime64[ns] 2091-01-01T12:00:00 ... 2100-12-31T12:00:00
    lon      (y, x) float64 dask.array<chunksize=(10, 14), meta=np.ndarray>
    lat      (y, x) float64 dask.array<chunksize=(10, 14), meta=np.ndarray>
  * x        (x) float64 10.18 10.34 10.5 10.66 10.82 ... 11.8 11.96 12.12 12.28
  * y        (y) float64 46.13 46.24 46.35 46.47 ... 46.8 46.92 47.03 47.14
Data variables:
    sfcWind  (time, y, x) float32 dask.array<chunksize=(3652, 10, 14), meta=np.ndarray>

# Conversion to pandas from xarray

In [11]:
# Step 5: Flatten Data
n_time_steps = data[0].shape[0]
n_lat, n_lon = data[0].shape[1], data[0].shape[2]
logger.info("for feature")
logger.info(f"{n_time_steps} , {n_lat}, {n_lon}")

n_time_steps = ndsi_labels.shape[0]
n_lat, n_lon = ndsi_labels.shape[1], ndsi_labels.shape[2]
logger.info("for label")
logger.info(f"{n_time_steps} , {n_lat}, {n_lon}")


2024-05-24 15:59:29,620 - INFO - for feature
2024-05-24 15:59:29,622 - INFO - 3652 , 10, 14
2024-05-24 15:59:29,625 - INFO - for label
2024-05-24 15:59:29,627 - INFO - 6518 , 10, 14


In [12]:
temp0 = to_array(data, 0)
temp1 = to_array(data, 1)
temp2 = to_array(data, 2)
temp3 = to_array(data, 3)
temp4 = to_array(data, 4)
temp5 = to_array(data, 5)


In [13]:
dates = ds["time"].values
lats = ndsi_ds["lat"].values
lons = ndsi_ds["lon"].values

#********** Dates *************
dates_array_to_append = []
for a in range(len(dates)):
    x = dates[a]
    for b in range(140):
        dates_array_to_append.append(x)
dates_array_to_append = np.array(dates_array_to_append)

#********** Latitude *************
lats_array_to_append = []
for c in range(len(dates)):
    for a in range(len(lats)):
        x = lats[a]
        for b in range(len(lons)):
            lats_array_to_append.append(x)
lats_array_to_append = np.array(lats_array_to_append)

#********** Longitude *************
lons_array_to_append = []
for b in range(len(dates)*len(lats)):
    for a in range(len(lons)):
        x = lons[a]
        lons_array_to_append.append(x)

lons_array_to_append = np.array(lons_array_to_append)


logger.info(f"Length of Dates: {len(dates_array_to_append)}")
logger.info(f"Length of Latitude: {len(lats_array_to_append)}")
logger.info(f"Length of Longitude: {len(lons_array_to_append)}")

2024-05-24 15:59:31,655 - INFO - Length of Dates: 511280
2024-05-24 15:59:31,656 - INFO - Length of Latitude: 511280
2024-05-24 15:59:31,657 - INFO - Length of Longitude: 511280


# Data cleaning, feature engineering with pandas

In [14]:
dict_temp = {
    "Date": dates_array_to_append,
    "Latitude": lats_array_to_append,
    "Longitude": lons_array_to_append,
    climate_vars[0]: temp0,
    climate_vars[1]: temp1,
    climate_vars[2]: temp2,
    climate_vars[3]: temp3,
    climate_vars[4]: temp4,
    climate_vars[5]: temp5
}


# Create the DataFrame
df = pd.DataFrame(dict_temp)
df['Latitude'] = df['Latitude'].round(2)
df['Longitude'] = df['Longitude'].round(2)

df, drop_nan = df.dropna(ignore_index=False), "yes"


In [15]:
# df = df.reset_index()
df["Month"] = df['Date'].dt.month
df['month_sin'] = np.sin(2 * np.pi * df['Month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['Month']/12)

df['week_number'] = df["Date"].dt.isocalendar().week

In [16]:
lats = np.unique(df.Latitude.values)
lons = np.unique(df.Longitude.values)

latitude_bins = divide_range(lats[0], lats[-1], divide_latitude_in_these_many_parts)   #10 values 
longitude_bins = divide_range(lons[0], lons[-1], divide_longitude_in_these_many_parts) #14 values

df['Latitude_Group'] = pd.cut(df['Latitude'], bins=latitude_bins, labels=False)
df['Longitude_Group'] = pd.cut(df['Longitude'], bins=longitude_bins, labels=False)

# Apply one-hot encoding
latitude_dummies = pd.get_dummies(df['Latitude_Group'], prefix='Latitude', dtype=int)
longitude_dummies = pd.get_dummies(df['Longitude_Group'], prefix='Longitude', dtype=int)

# Concatenate one-hot encoded columns with the original DataFrame
df = pd.concat([df, latitude_dummies, longitude_dummies], axis=1)
temp = df
df = df.drop(["Latitude_Group",	"Longitude_Group"], axis =1)

print_info_about_latitude_longitude_groups(temp)

2024-05-24 15:59:35,146 - INFO - Latitude Group
2024-05-24 15:59:35,148 - INFO - {
 0: [46.13 46.24],
 1: [46.35 46.47],
 2: [46.58 46.69],
 3: [46.8  46.92],
 4: [47.03 47.14]
}
2024-05-24 15:59:35,150 - INFO - Longitude Group
2024-05-24 15:59:35,153 - INFO - {
 0: [10.18 10.34 10.5 ],
 1: [10.66 10.82],
 2: [10.99 11.15],
 3: [11.31 11.47],
 4: [11.64 11.8 ],
 5: [11.96 12.12 12.28]
}


In [27]:
print(df.columns)

Index(['Date', 'Latitude', 'Longitude', 'tas', 'pr', 'hurs', 'psl', 'rsds',
       'sfcWind', 'Month', 'month_sin', 'month_cos', 'week_number',
       'Latitude_0', 'Latitude_1', 'Latitude_2', 'Latitude_3', 'Latitude_4',
       'Longitude_0', 'Longitude_1', 'Longitude_2', 'Longitude_3',
       'Longitude_4', 'Longitude_5'],
      dtype='object')


In [28]:
df= df.drop(["Month", "week_number"],axis =1)

In [29]:
parameter_array = ["tas", "pr", "hurs", "psl",  "rsds", "sfcWind"]
min_max_df = pd.read_csv("./min_max_of_all_parameters_2.csv")
for parameter in parameter_array:
    min_val, max_val =  get_min_max(min_max_df, parameter)
    df[parameter] = (df[parameter] - min_val) / (max_val - min_val)

In [30]:
df_insurance = df

In [31]:
# b_array = [0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99]  # Desired number of zero values
zero_values_array = []
b_array = percentage_of_data_in_summer_months
for b in b_array:
    
    df = df_insurance
    df["Month"] = df['Date'].dt.month
    df_to_delete = df
    df = df[df['Month'].isin([1, 2, 3, 12])]
    logger.info(f"Number of datapoints in Original df: {len(df_to_delete)}")

    # Step 6: Display the final value of b
    logger.info(f"Value of b: {b}")

    # Now you can use this final value of b to sample your DataFrame
    final_sampled_data = pd.concat([df_to_delete[(df_to_delete['Date'].dt.month == month)].sample(frac=b, random_state=1) for month in range(4, 12)])
    df = pd.concat([final_sampled_data, df], ignore_index=False)
    df = df.sort_index()
    logger.info(f"Number of datapoints after sampling df: {len(df)}")


2024-05-24 12:20:40,867 - INFO - Number of datapoints in Original df: 511280
2024-05-24 12:20:40,869 - INFO - Value of b: 0.4


2024-05-24 12:20:41,507 - INFO - Number of datapoints after sampling df: 306320


In [32]:
if percentage == 100:
    df.to_csv(f"./ig_complete_df_{scenario}{future_type}.csv")

#bookmark2

# Prediction

In [33]:
scenario_array = ["rcp26", "rcp45", "rcp85"]
# scenario_array = ["rcp85"]
prediction_array = []

for scenario in scenario_array:
    df = pd.read_csv(f"./ig_stuff/ig_complete_df_{scenario}{future_type}.csv")
    df = df.sample(frac = percentage/100, random_state=1)
    df = df.sort_index()
    logger.info(f"Number of rows: {len(df)}")

    X_og= df.drop(["Date", "Latitude", "Longitude", "Month", "Unnamed: 0"], axis=1).values
    temp_shape = X_og.shape[1]
    X = X_og.reshape(-1, 1, temp_shape)  # Reshaping to (146246, 1, 32)
    logger.info(f"X shape is: {X.shape}")

    # Check for GPU availability
    if tf.config.list_physical_devices('GPU'):
        logger.info('GPU found. Running on GPU.')
    else:
        logger.info('No GPU found. Running on CPU.')

    # Specify GPU device if available
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)

    # Convert data to TensorFlow tensors
    X_tensor = tf.convert_to_tensor(X, dtype=tf.float32)
    model = load_model(rf'.\model_runs\saved_run\model.h5')

    y = model.predict(X_tensor)
    remove_eta_lines(run_number)
    prediction_array.append(y)

prediction_array = np.array(prediction_array)

2024-05-24 12:21:12,264 - INFO - Number of rows: 3063
2024-05-24 12:21:12,266 - INFO - X shape is: (3063, 1, 19)
2024-05-24 12:21:12,329 - INFO - GPU found. Running on GPU.


96/96 [==============================] - 5s 10ms/step


2024-05-24 12:21:21,177 - INFO - Number of rows: 3063
2024-05-24 12:21:21,181 - INFO - X shape is: (3063, 1, 19)
2024-05-24 12:21:21,182 - INFO - GPU found. Running on GPU.


96/96 [==============================] - 2s 7ms/step


2024-05-24 12:21:26,814 - INFO - Number of rows: 3063
2024-05-24 12:21:26,820 - INFO - X shape is: (3063, 1, 19)
2024-05-24 12:21:26,822 - INFO - GPU found. Running on GPU.


96/96 [==============================] - 2s 7ms/step


# Plotting

In [46]:
comparison_df = DataFrame(df, scenario_array, prediction_array)

comparison_df['Date'] = pd.to_datetime(comparison_df['Date'])

# df_image_path = os.path.join(folder_name, 'val_vs_predicted.csv')
comparison_df.to_csv(f"./ig_stuff/ig{future_type}_predictions.csv")
comparison_df

,Date,Latitude,Longitude,rcp26,rcp45,rcp85
0,2091-01-01 12:00:00,46.24,10.99,0.452088,0.390061,0.360785
1,2091-01-01 12:00:00,46.80,10.50,0.672864,0.593397,0.567339
2,2091-01-01 12:00:00,47.14,11.96,0.684732,0.609488,0.571064
3,2091-01-02 12:00:00,47.03,11.15,0.660951,0.597581,0.620353
4,2091-01-03 12:00:00,46.24,11.31,0.418256,0.317614,0.150433
...,...,...,...,...,...,...
3058,2100-12-29 12:00:00,46.24,12.12,0.610957,0.181218,0.176598
3059,2100-12-30 12:00:00,46.35,11.15,0.496198,0.196980,0.136735
3060,2100-12-30 12:00:00,46.58,11.31,0.612554,0.214572,0.140660
3061,2100-12-30 12:00:00,47.14,10.34,0.592814,0.548320,0.551393


In [36]:
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  46.58) & (comparison_df['Longitude'] == 11.15)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Centre without mean"
fig = px.line(comparison_df_1, x="Date" , y = scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, 'Centre_without_mean.png')
fig.write_image(plotly_image_path)

fig

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


In [37]:
# top left
temp_lat =  lats[-3]
temp_lon = lons[2]
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  temp_lat) & (comparison_df['Longitude'] == temp_lon)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Top left"
fig = px.line(comparison_df_1, x="Date" , y = scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)

fig

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [38]:
# top right
temp_lat =  lats[-2]
temp_lon = lons[-2]
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  temp_lat) & (comparison_df['Longitude'] == temp_lon)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Top Right"
fig = px.line(comparison_df_1, x="Date" , y = scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)

fig

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [39]:
# bottom left
temp_lat =  lats[1]
temp_lon = lons[2]
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  temp_lat) & (comparison_df['Longitude'] == temp_lon)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Bottom left"
fig = px.line(comparison_df_1, x="Date" , y = scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)

fig

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [40]:
# bottom right
temp_lat =  lats[1]
temp_lon = lons[-4]
comparison_df_1 = comparison_df[(comparison_df['Latitude'] ==  temp_lat) & (comparison_df['Longitude'] == temp_lon)]
comparison_df_1, scale = comparison_df_1.sort_values(by='Date'), "Bottom right"
fig = px.line(comparison_df_1, x="Date" , y = scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)

fig

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [41]:
comparison_df2 = comparison_df
comparison_df2['Date'] = pd.to_datetime(comparison_df2['Date'], dayfirst=True)
comparison_df2['Date'], scale = comparison_df2['Date'].dt.strftime('%d-%m-%Y'), "Average Daily"
comparison_df2= comparison_df2.groupby('Date')[scenario_array].mean().reset_index()
comparison_df2['Date'] = pd.to_datetime(comparison_df2['Date'], format='%d-%m-%Y')
comparison_df2 = comparison_df2.sort_values(by='Date')
# comparison_df2
fig = px.scatter(comparison_df2, x="Date" , y=scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)
fig

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [42]:
fig = px.line(comparison_df2, x="Date" , y=scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)
fig

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [43]:
comparison_df_3 = comparison_df
comparison_df_3['Date'] = pd.to_datetime(comparison_df_3['Date'], dayfirst=True)
comparison_df_3['Date'], scale = comparison_df_3['Date'].dt.strftime('%m-%Y'), "Average monthly"
comparison_df_3= comparison_df_3.groupby('Date')[scenario_array].mean().reset_index()
comparison_df_3['Date'] = pd.to_datetime(comparison_df_3['Date'])
comparison_df_3 = comparison_df_3.sort_values(by='Date')

fig = px.line(comparison_df_3, x="Date" , y=scenario_array)
fig.update_layout(title=f'{scenario_array} | {scale}', width = 1800, height = 400)

plotly_image_path = os.path.join(folder_name, f"{scale}.png")
fig.write_image(plotly_image_path)

fig

C:\Users\suvam\AppData\Local\Temp\ipykernel_32976\1191913206.py:5: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

c:\Users\suvam\miniconda3\envs\tensorflow_env\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

